# CatBoost

[Prokhorenkova et al. (2018). CatBoost: unbiased boosting with categorical features](https://arxiv.org/pdf/1706.09516.pdf)

## Prediction Shift

[CatBoost論文のprediction shiftについて完全に理解する - threecourse’s blog](https://threecourse.hatenablog.com/entry/2020/05/29/204944)

- 各iterationで同じ訓練データを使うことの問題
- 1本目と2本目の木の作成で同じデータセットを使うことにより、予測値の期待値に$1 / (n − 1)$ に比例する偏りが発生する。
- prediction shiftの影響を受けないよう、CatBoostではデータ数が少ない場合にはordered boostingというアルゴリズムを使用するようになっている。